###### 【問題1】機械翻訳の実行とコードリーディング
Keras公式のサンプルコードで、短い英語からフランス語への変換が行えるのでこれを動かしてください。
その上でこのサンプルコードの各部分がどういった役割かを読み取り、まとめてください。以下のようにどこからどこの行が何をしているかを記述してください。

エンコーダー部を定義。encoderの出力は実際は使わず、encoder_statesのみをdecoderへ渡す。

In [ ]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

デコーダー部分を定義。出力層のsoftmaxへ入力するように全結合をする。

In [ ]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

定義したエンコーダ、デコーダを渡して学習モデルを定義。更新式はAdagardを改良したRMSPROPを用いている。Lossは多クラス分類なのでcategorical_crossentropy

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

推論に使うモデルを定義。学習モデルではデコーダ部分のLSTMへの入力はtargetデータであったが、推論モデルでは、1シーケンス前のsoftmaxで推測された結果が次のLSTMヘ入力される。

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

上で定義したモデルを使ってデコードを行う。

In [ ]:
def decode_sequence(input_seq):

###### 学習結果

<img src="seq2seq/Learning.png">

<img src="seq2seq/prediction.png">

###### 【問題2】イメージキャプショニングの学習済みモデルの実行
上記実装において5. Test the modelの項目を実行してください。また、自身で用意した画像に対しても文章を生成してください。これらに対してどういった文章が出力されたかを記録して提出してください。
データセットからの学習は行わず、学習済みの重みをダウンロードして利用します。
注意点として、デフォルトで設定されている重みのファイル名と、ダウンロードできる重みのファイル名は異なっています。ここは書き換える必要があります。

<img src="image-caption/sakana_kun.png">

<img src="image-caption/caption.png">

正しい表現：男、帽子  
間違った表現：ネクタイ、テーブルの上

###### 【問題3】Kerasで動かしたい場合はどうするかを調査
PyTorchによる実装を動かしましたが、何らかの理由からKerasで動かしたい状況が考えられます。どういった手順を踏むことになるか調査し、できるだけ詳しく説明してください。
特に今回はPyTorchのための学習済みの重みをKerasで使えるようにしたいので、その点については必ず触れてください。

1.PytorchからKerasへコードの変更  
2.mmdmmなどのツールを使いPytorchの学習モデルをKerasへ変更する。（https://github.com/Microsoft/MMdnn）  
以下の図のように１度PytorchモデルをIRという中間ファイルへ変換し、それを目的のモデルへ変換（今回はkerasへ）する。

<img src="image-caption/mmdnn_fig.png">


3.書き換えたモデルで再度学習

###### 【問題4】（アドバンス課題）コードリーディングと書き換え
モデル部分はmodel.pyに書かれていますが、Kerasではこのモデルがどのように記述できるかを考え、コーディングしてください。その際機械翻訳のサンプルコードが参考になります。

###### Original Pytorch

In [ ]:
class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        """Load the pretrained ResNet-152 and replace top fc layer."""
        super(EncoderCNN, self).__init__()
        # ImageNet
        resnet = models.resnet152(pretrained=True)  
        # delete the last fc layer.
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        #全結合でVectorizeされた文字データと同じ次元に調整
        self.linear = nn.Linear(resnet.fc.in_features, embed_size)
        self.bn = nn.BatchNorm1d(embed_size, momentum=0.01)
        
    def forward(self, images):
        """Extract feature vectors from input images."""
        # 層の凍結 imagenetは学習しない
        with torch.no_grad():
            features = self.resnet(images)
        #Flatten
        features = features.reshape(features.size(0), -1)
        features = self.bn(self.linear(features))
        return features

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, max_seq_length=20):
        """Set the hyper-parameters and build the layers."""
        super(DecoderRNN, self).__init__()
        #文字のVectorize
        self.embed = nn.Embedding(vocab_size, embed_size)
        #embed_size=(seq, batch, feature)　Hidden_size:(node,node) num_layer:(Vocabの数 LSTMの繰り返し回数)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        #LSTMの出力をSoftmaxに入れるためにshape調整
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.max_seg_length = max_seq_length
        
    #training
    def forward(self, features, captions, lengths):
        """Decode image feature vectors and generates captions."""
        #正解データ(caption)をベクトル変換
        embeddings = self.embed(captions)
        #エンコーダで抽出した画像の特徴量とベクトル化したcaptionをconcat
        embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True) 
        #LSTMへ入力
        hiddens, _ = self.lstm(packed)
        #softmaxへの入力
        outputs = self.linear(hiddens[0])
        return outputs
    
    #Prediction
    def sample(self, features, states=None):
        """Generate captions for given image features using greedy search."""
        sampled_ids = []
        inputs = features.unsqueeze(1)
        for i in range(self.max_seg_length):
            hiddens, states = self.lstm(inputs, states)          # hiddens: (batch_size, 1, hidden_size)
            outputs = self.linear(hiddens.squeeze(1))            # outputs:  (batch_size, vocab_size)
            _, predicted = outputs.max(1)                        # predicted: (batch_size)
            sampled_ids.append(predicted)
            inputs = self.embed(predicted)                       # inputs: (batch_size, embed_size)
            inputs = inputs.unsqueeze(1)                         # inputs: (batch_size, 1, embed_size)
        sampled_ids = torch.stack(sampled_ids, 1)                # sampled_ids: (batch_size, max_seq_length)
        return sampled_ids

###### Keras

In [20]:
from keras.models import Model
from keras.layers import Dense, Reshape, Embedding, Concatenate, LSTM, Input, Activation
from keras.applications.resnet50 import ResNet50
import tensorflow as tf

emb_size = 256
num_of_words = 100
hidden_size = 32
input_size =(256,256,3)

"""
image_net = ResNet50(
   weights='imagenet',
   include_top=False,
   input_shape=(224,224,3), trainable=False, pooling="ave")
"""
#Encoder
input_tensor = Input(shape=input_size)
image_net = ResNet50(include_top=False, weights='imagenet', input_shape=input_size, pooling='avg')
x = image_net(input_tensor)
x = Dense(emb_size)(x)
x = Reshape((1,emb_size))(x)

#target caption
target_caption = Input(shape=(None,))
y = Embedding(num_of_words, emb_size)(target_caption)
y = Concatenate(1)([x, y])
#Decoder
output = LSTM(hidden_size, return_sequences=True)(y)
output = Dense(num_of_words)(output)
output = Activation('softmax')(output)

model = Model([input_tensor, target_caption], output)
model.compile(optimizer=tf.train.AdamOptimizer(0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

###### 【問題5】（アドバンス課題）発展的調査
他の言語の翻訳を行う場合は？
問題1の実装を使い日本語と英語の翻訳を行いたい場合はどのような手順を踏むか考えてみましょう。
機械翻訳の発展的手法にはどのようなものがある？  
機械翻訳のための発展的手法にはどういったものがあるか調査してみましょう。  
文章から画像生成するには？  
イメージキャプショニングとは逆に文章から画像を生成する手法もあります。どういったものがあるか調査してみましょう。